# 神奈川県のコロナデータ分析

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
sns.set_style(style="da", rc=None)
sns.set(font_scale = 1.2)

import japanize_matplotlib

ValueError: style must be one of white, dark, whitegrid, darkgrid, ticks

In [ ]:
import sys
sys.path.append('../')
%cd '../'

import src.tool as tool

%reload_ext autoreload
%autoreload 2

In [ ]:
patients = tool.read_patients()

In [ ]:
patients.tail(5)

In [ ]:
patients = patients.drop(["備考", "情報元"], axis=1)
patients["id"] = patients.index

## 時間別感染者数

### 日別

In [ ]:
groupby = patients.groupby('発病確認日')

In [ ]:
df_by_day = pd.DataFrame()
df_by_day["感染者数"] = groupby["id"].count()

In [ ]:
df_by_day

### 累計

In [ ]:
##

### 週別

In [ ]:
##

### 月別

## 属性別感染者数

### 地域別

In [ ]:
area = patients
area["居住地"] = patients["居住地"].apply(tool.cut_ku)
area_df = area.groupby("居住地").count().sort_values("id", ascending=False)["id"].to_frame()
area_df.rename(columns={"id": "感染者数"}, inplace=True)

In [ ]:
area_df

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.countplot(x="居住地", data=area, order=area['居住地'].value_counts().index)

ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize=14)
ax.set_xlabel('居住地',fontsize=14)
ax.set_ylabel('感染者数',fontsize=14)

plt.tight_layout()
plt.show()

### 年代別

In [ ]:
age = patients
age_df = age.groupby("年代").count()["id"].to_frame()
age_df.rename(columns={"id": "感染者数"}, inplace=True)

In [ ]:
age_df

In [ ]:
plt.figure(figsize=(12, 4))
ax = sns.countplot(x="年代", data=age, order=["10代", "20代", "30代", "40代", "50代", "60代", "70代", "80代", "90代"])

ax.set_xticklabels(ax.get_xticklabels(), fontsize=14)

ax.set_xlabel('年代',fontsize=14)
ax.set_ylabel('感染者数',fontsize=14)

plt.tight_layout()
plt.show()

### 男女別

In [ ]:
sex_df = patients
sex_df = sex_df.groupby("性別").count()["id"].to_frame()
sex_df.rename(columns={"id": "感染者数"}, inplace=True)

In [ ]:
sex_df